In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.legacy import data
from torchtext.legacy import datasets

import random
import numpy as np

import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

tokenizer = nltk.RegexpTokenizer(r"\w+").tokenize

TEXT = data.Field(tokenize=tokenizer, lower = True)
LABEL = data.LabelField()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
train_data, valid_data, test_data = datasets.SNLI.splits(TEXT, LABEL)

downloading snli_1.0.zip


snli_1.0.zip: 100%|██████████| 94.6M/94.6M [00:09<00:00, 9.68MB/s]


extracting


In [5]:
print(type(train_data))
print(type(valid_data))
print(type(test_data))

<class 'torchtext.legacy.datasets.nli.SNLI'>
<class 'torchtext.legacy.datasets.nli.SNLI'>
<class 'torchtext.legacy.datasets.nli.SNLI'>


In [6]:

print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 549367
Number of validation examples: 9842
Number of testing examples: 9824


In [7]:
print(vars(train_data.examples[0]))

{'premise': ['a', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane'], 'hypothesis': ['a', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition'], 'label': 'neutral'}


In [8]:
MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ,
                 vectors = "glove.6B.300d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:42, 5.30MB/s]                           
100%|█████████▉| 399903/400000 [00:54<00:00, 7151.54it/s]

In [9]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")

Unique tokens in TEXT vocabulary: 23352


In [10]:
print(TEXT.vocab.freqs.most_common(20))

[('a', 1439104), ('the', 534701), ('in', 407300), ('is', 373355), ('man', 266237), ('on', 235910), ('and', 206376), ('are', 199024), ('of', 192432), ('with', 169236), ('woman', 137630), ('two', 122259), ('people', 121159), ('to', 113972), ('at', 98662), ('wearing', 81024), ('an', 80212), ('his', 72467), ('shirt', 62744), ('young', 61515)]


In [11]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'a', 'the', 'in', 'is', 'man', 'on', 'and', 'are']


In [12]:
print(LABEL.vocab.itos)

['entailment', 'contradiction', 'neutral']


In [13]:
BATCH_SIZE = 512

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [14]:
class NLIBiLSTM(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim,
                 hidden_dim,
                 n_lstm_layers,
                 n_fc_layers,
                 output_dim, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
                                
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.translation = nn.Linear(embedding_dim, hidden_dim)
        
        self.lstm = nn.LSTM(hidden_dim, 
                            hidden_dim, 
                            num_layers = n_lstm_layers, 
                            bidirectional = True, 
                            dropout=dropout if n_lstm_layers > 1 else 0)
        
        fc_dim = hidden_dim * 2
        
        fcs = [nn.Linear(fc_dim * 2, fc_dim * 2) for _ in range(n_fc_layers)]
        
        self.fcs = nn.ModuleList(fcs)
        
        self.fc_out = nn.Linear(fc_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, prem, hypo):

        prem_seq_len, batch_size = prem.shape
        hypo_seq_len, _ = hypo.shape
        
        #prem = [prem sent len, batch size]
        #hypo = [hypo sent len, batch size]
        
        embedded_prem = self.embedding(prem)
        embedded_hypo = self.embedding(hypo)
        
        #embedded_prem = [prem sent len, batch size, embedding dim]
        #embedded_hypo = [hypo sent len, batch size, embedding dim]
        
        translated_prem = F.relu(self.translation(embedded_prem))
        translated_hypo = F.relu(self.translation(embedded_hypo))
        
        #translated_prem = [prem sent len, batch size, hidden dim]
        #translated_hypo = [hypo sent len, batch size, hidden dim]
        
        outputs_prem, (hidden_prem, cell_prem) = self.lstm(translated_prem)
        outputs_hypo, (hidden_hypo, cell_hypo) = self.lstm(translated_hypo)

        #outputs_x = [sent len, batch size, n directions * hid dim]
        #hidden_x = [n layers * n directions, batch size, hid dim]
        #cell_x = [n layers * n directions, batch size, hid dim]
        
        hidden_prem = torch.cat((hidden_prem[-1], hidden_prem[-2]), dim=-1)
        hidden_hypo = torch.cat((hidden_hypo[-1], hidden_hypo[-2]), dim=-1)
        
        #hidden_x = [batch size, fc dim]

        hidden = torch.cat((hidden_prem, hidden_hypo), dim=1)

        #hidden = [batch size, fc dim * 2]
            
        for fc in self.fcs:
            hidden = fc(hidden)
            hidden = F.relu(hidden)
            hidden = self.dropout(hidden)
        
        prediction = self.fc_out(hidden)
        
        #prediction = [batch size, output dim]
        
        return prediction

In [15]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 300
N_LSTM_LAYERS = 2
N_FC_LAYERS = 3
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = NLIBiLSTM(INPUT_DIM,
                  EMBEDDING_DIM,
                  HIDDEN_DIM,
                  N_LSTM_LAYERS,
                  N_FC_LAYERS,
                  OUTPUT_DIM,
                  DROPOUT,
                  PAD_IDX)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 15,032,703 trainable parameters


In [17]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([23352, 300])


In [18]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [-0.2971,  0.0940, -0.0967,  ...,  0.0597, -0.2285,  0.2960],
        ...,
        [-0.2149,  0.0846, -0.2949,  ...,  0.2379,  0.4804, -0.3348],
        [-0.0626, -0.3148, -0.9665,  ...,  1.3171, -0.8476, -0.4057],
        [ 0.3580, -0.0304,  0.3355,  ...,  0.0703, -0.5158,  0.1819]])

In [19]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2971,  0.0940, -0.0967,  ...,  0.0597, -0.2285,  0.2960],
        ...,
        [-0.2149,  0.0846, -0.2949,  ...,  0.2379,  0.4804, -0.3348],
        [-0.0626, -0.3148, -0.9665,  ...,  1.3171, -0.8476, -0.4057],
        [ 0.3580, -0.0304,  0.3355,  ...,  0.0703, -0.5158,  0.1819]])


In [20]:

model.embedding.weight.requires_grad = False

In [21]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,027,103 trainable parameters


In [22]:
optimizer = optim.Adam(model.parameters())

In [23]:
criterion = nn.CrossEntropyLoss()

In [24]:
model = model.to(device)
criterion = criterion.to(device)

In [25]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #preds = preds.to(device)
    #y = y.to(device)
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    #max_preds = max_preds.to(device)
    correct = (max_preds.squeeze(1)==y).float()
    #correct = correct.to(device)
    return correct.sum() / len(y)

In [26]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        prem = batch.premise
        hypo = batch.hypothesis
        labels = batch.label
        
        optimizer.zero_grad()
        
        #prem = [prem sent len, batch size]
        #hypo = [hypo sent len, batch size]
        
        predictions = model(prem, hypo)
        
        #predictions = [batch size, output dim]
        #labels = [batch size]
        
        loss = criterion(predictions, labels)
                
        acc = categorical_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [27]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            prem = batch.premise
            hypo = batch.hypothesis
            labels = batch.label
                        
            predictions = model(prem, hypo)
            
            loss = criterion(predictions, labels)
                
            acc = categorical_accuracy(predictions, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [37]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 9m 24s
	Train Loss: 0.469 | Train Acc: 81.61%
	 Val. Loss: 0.521 |  Val. Acc: 79.61%
Epoch: 02 | Epoch Time: 9m 25s
	Train Loss: 0.442 | Train Acc: 82.77%
	 Val. Loss: 0.512 |  Val. Acc: 79.64%
Epoch: 03 | Epoch Time: 9m 24s
	Train Loss: 0.415 | Train Acc: 83.96%
	 Val. Loss: 0.514 |  Val. Acc: 80.12%
Epoch: 04 | Epoch Time: 9m 25s
	Train Loss: 0.389 | Train Acc: 85.05%
	 Val. Loss: 0.527 |  Val. Acc: 80.18%
Epoch: 05 | Epoch Time: 9m 25s
	Train Loss: 0.362 | Train Acc: 86.19%
	 Val. Loss: 0.545 |  Val. Acc: 79.92%


In [38]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.518 |  Test Acc: 79.65%


In [39]:
def predict_inference(premise, hypothesis, text_field, label_field, model, device):
    
    model.eval()
    
    if isinstance(premise, str):
        premise = text_field.tokenize(premise)
    
    if isinstance(hypothesis, str):
        hypothesis = text_field.tokenize(hypothesis)
    
    if text_field.lower:
        premise = [t.lower() for t in premise]
        hypothesis = [t.lower() for t in hypothesis]
        
    premise = [text_field.vocab.stoi[t] for t in premise]
    hypothesis = [text_field.vocab.stoi[t] for t in hypothesis]
    
    premise = torch.LongTensor(premise).unsqueeze(1).to(device)
    hypothesis = torch.LongTensor(hypothesis).unsqueeze(1).to(device)
    
    prediction = model(premise, hypothesis)
    
    prediction = prediction.argmax(dim=-1).item()
    
    return label_field.vocab.itos[prediction]

In [40]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a woman sitting on a green bench.'

predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'contradiction'

In [41]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a man sitting on a blue bench.'

predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'entailment'

In [42]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a male sat on a lime bench.'

predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'entailment'

In [43]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a person on a park bench'

predict_inference(premise, hypothesis, TEXT, LABEL, model, device)

'neutral'